In [1]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models import doc2vec
import re
import pandas as pd
from numba import jit, cuda
@jit
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in tqdm(enumerate(corpus)):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

C:\Users\Jerry\AppData\Local\Temp\ipykernel_12336\1549811410.py:11: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def label_sentences(corpus, label_type):


In [2]:
@jit
def read_data():
    test_csv = pd.read_csv('datasets/cleaned_test_data.csv')
    train_csv = pd.read_csv('datasets/cleaned_trained_data.csv')
    test_csv = test_csv.drop(test_csv.index[0])
    train_csv = train_csv.drop(train_csv.index[0])
    return train_csv,test_csv
train_data,test_data = read_data()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_12336\3652498904.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def read_data():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_12336\3652498904.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "read_data" failed type inference due to: Unknown attribute 'read_csv' of type Module(<module 'pandas' from 'c:\\Users\\Jerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pandas\\__init__.py'>)

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_12336\3652498904.py", line 3:
<source missing, REPL/exec in use?>

During: typing of get attribute at C:\Users\Jerry\Ap

In [4]:
x_train_data = label_sentences(train_data['text'],'Train')
x_test_data = label_sentences(test_data['text'],'Test')
all_data = x_train_data + x_test_data

C:\Users\Jerry\AppData\Local\Temp\ipykernel_27428\1549811410.py:10: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "label_sentences" failed type inference due to: Untyped global name 'tqdm': Cannot determine Numba type of <class 'type'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_27428\1549811410.py", line 18:
<source missing, REPL/exec in use?>

  @jit
C:\Users\Jerry\AppData\Local\Temp\ipykernel_27428\1549811410.py:10: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "label_sentences" failed type inference due to: Untyped global name 'tqdm': Cannot determine Numba type of <class 'type'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_27428\1549811410.py", line 18:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "label_sentences" was compiled 

In [6]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

@jit
def preprocessData():
    for epoch in range(1):
        print(epoch)
        model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
        model_dbow.alpha -= 0.002
        model_dbow.min_alpha = model_dbow.alpha
preprocessData()

100%|██████████| 801219/801219 [00:00<00:00, 1657312.65it/s]


KeyboardInterrupt: 

In [5]:

import numpy as np
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in tqdm(range(0, corpus_size)):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(x_train_data), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(x_test_data), 300, 'Test')

  0%|          | 0/768325 [00:00<?, ?it/s]C:\Users\Jerry\AppData\Local\Temp\ipykernel_24028\2779434855.py:14: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i] = model.docvecs[prefix]
100%|██████████| 32894/32894 [00:02<00:00, 16356.95it/s]


In [18]:
categories = {'Society & Culture' :1,'Science & Mathematics':2,'Health':3, 'Education & Reference':4,
            'Computers & Internet' :5,'Sports' :6,'Business & Finance' :7,'Entertainment & Music' : 8,
            'Family & Relationships':9, 'Politics & Government':10}
train_data['class']=train_data['class'].map(categories)

In [9]:
my_tags = [1,2,3,4,5,6,7,8,9,10]

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# @jit(target_backend='cuda')
def buildModel():
    logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=10000)
    logreg = logreg.fit(train_vectors_dbow, train_data['class'])
    y_pred = logreg.predict(test_vectors_dbow)
    print('accuracy %s' % accuracy_score(y_pred, test_data['class']))
# print(classification_report(test_data['class'], y_pred,target_names=my_tags))
logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=10000)
logreg = logreg.fit(train_vectors_dbow, train_data['class'])
y_pred = logreg.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, test_data['class']))
# buildModel()
y_pred

accuracy 0.0


array([9, 2, 1, ..., 9, 1, 5], dtype=int64)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
@jit
def buildModel():
    logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                ])
    logreg.fit(train_data['text'], train_data['class'])
    y_pred = logreg.predict(test_data['text'])
    print('accuracy %s' % accuracy_score(y_pred, test_data['class']))
    # print(classification_report(test_data['class'], y_pred,target_names=my_tags))
buildModel()

C:\Users\Jerry\AppData\Local\Temp\ipykernel_12336\982528151.py:8: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def buildModel():
C:\Users\Jerry\AppData\Local\Temp\ipykernel_12336\982528151.py:7: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "buildModel" failed type inference due to: Untyped global name 'Pipeline': Cannot determine Numba type of <class 'abc.ABCMeta'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_12336\982528151.py", line 9:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Jerry\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function 

In [3]:

from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(train_data['text'], train_data['class'])

y_pred = sgd.predict(test_data['text'])

print('accuracy %s' % accuracy_score(y_pred, test_data['class']))


NameError: name 'Pipeline' is not defined

In [24]:
test_data['y_pred']=y_pred
test_data

,text,class,y_pred
1,makes friendship click ? spark keep going ? go...,9,9
2,Zebras stripes ? purpose stripes ? serve Zebra...,2,2
3,"animals sense humour ? example , would dog dig...",2,1
4,website HotorNot ? think website ? anyone met ...,9,9
5,take photos e-commerce ? need take photos onli...,7,5
...,...,...,...
32890,conservatives want us stay pay Iraq ? Cant see...,10,10
32891,else thinks ref ( uru ) reprimanded ! ? ! ? ! ...,6,6
32892,guys pleez ? anyone 4 13to15 want talk dont wa...,9,9
32893,Tell something life people n't know ....... ? ...,8,1
